In [38]:
# NER 前處理(長度限制) new share
# NES = not entity sentence
# Fix Length = FL
import random
import re
import pandas as pd
import json
import os, csv
import numpy as np
from operator import itemgetter
model="w2ner"
data_format = "test"        #dev, train/test
file_format = "cadec"       #share13/share14/cadec
ans_data_path = "test"      #dev, train /test

#share13
# from_enhance_count =   "model_share14_new_split_ori_our_area"
# from_enhance_count =  "enhance_model_share14_our_split_12_look_both_side_ES-dynamic_size-F3-B3"
# from_enhance_count = "enhance_model_share14_our_split_14_inner_look_both_side_ES-dynamic_size-F3-B3"
# model_share14_new_split_ori_our_area/model_share14_new_split_ori_dai_area
to_enhance_count = "enhance_model_cadec_our_split_1_new_inner"
split_type="our"
#enhance_share13_new_origin_1/enhance_share13_share13_origin_1
model="w2ner"
if "cadec" in file_format:
    data_format = data_format        #dev, train/test
    file_format = 'cadec'      #share13/share14/cadec
    ans_data_path = data_format      #dev, train /test
    answer_file='acl/cadec/ann'
    folder_path = 'acl/'+file_format+'/text'
    # path = "acl/data/cadec/new_"+file_format+"/"+file_format+"_"+data_format+".json"
    path = "acl/"+file_format+"/"+file_format+"_word_list.json"
    pattern = r'[\w]+|[^\s\w]'
    pattern_s = r'[\w]+|\n|[\s]+|[^\s\w]'
else:
    data_format = data_format        #dev, train/test
    if "share13" in file_format:
        file_format = 'share13'       #share13/share14/cadec
    else:
        file_format = 'share14'      #share13/share14/cadec

    if ans_data_path=="test":
        ans_data_path=ans_data_path
    else:
        ans_data_path="train"
    answer_file='D:/hw/W2NER-final/acl/data/'+file_format+'/'+ans_data_path+'.ann'
    folder_path = 'D:/hw/W2NER-final/acl/data/'+file_format+'/'+ans_data_path+'/text'
    if "our" in split_type:
        # our pattern 
        pattern = r'[\w]+|[^\s\w]'
        pattern_s = r'[\w]+|\n|[\s]+|[^\s\w]'
        # path = "D:/hw/W2NER-final/acl/data/cadec/new_"+file_format+"/"+file_format+"_"+data_format+".json"
        path = "acl/"+file_format+"/"+file_format+"_word_list.json"
    if "dai" in split_type:
        # new pattern - dai
        contractions = set(["n't", "'s", "'ve", "'re", "'ll", "'d", "'m"])
        contractions |= set([x.replace("'", "’") for x in contractions])

        contractions_pattern_1 = "\w+(?="+")|\w+(?=".join(list(contractions))+")"
        contractions_pattern_2 = "|".join(list(contractions))

        pattern = "\w+(?=n't)|~\w+|"+contractions_pattern_1+"|\w+#\w*|\w+=\w*|\w+‘\w*|\w+“\w*|"+contractions_pattern_2+"|"+r"""[\w\d]+[*]+|[|]+|\w+"\w+|\w+'\w+|[\w]+|[^\s\w]"""
        pattern_s = "\w+(?=n't)|~\w+|"+contractions_pattern_1+"|\w+#\w*|\w+=\w*|\w+‘\w*|\w+“\w*|"+contractions_pattern_2+"|"+r"""[\w\d]+[*]+|[|]+|\w+"\w+|\w+'\w+|[\w]+|[^\s\w]"""+"|\n|[\s]+"
        path = "D:/hw/W2NER-final/acl/data/cadec/new_"+file_format+"/"+file_format+"_"+data_format+"_new.json"
    


ann_own_path = "ann"  #ann_vik / ann


# #cadec
# answer_file='D:/acl/data/cadec/adr/ann'
#share13/14
ans = pd.read_csv(answer_file, names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
ans_tmp = ans.groupby(['file_id'])
sentence_length = []
enhance_sentence_length = []

# use ground true answer
# ans_split = pd.read_csv(answer_file, names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, quoting=csv.QUOTE_NONE, na_values=['NaN', 'null'], delimiter='\t')

# use predict answer
# ans_split = pd.read_csv('ann/'+file_format+'/'+ann_own_path+'/w2ner_'+data_format+"_"+from_enhance_count+'.ann', names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, quoting=csv.QUOTE_NONE, na_values=['NaN', 'null'], delimiter='\t')

# ans_split = pd.read_csv('D:/hw/W2NER-final/ann/share13_our_split_12_look_both_side_ES-dynamic_size-F4-B4/test/w2ner_test_0.ann', names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, quoting=csv.QUOTE_NONE, na_values=['NaN', 'null'], delimiter='\t')
# ans_split = pd.read_csv(r"D:\hw\TOE-final\ann\cadec_19\test\w2ner_test_0.ann", names=["file_id", "label", "start_end", "entity", "other","entity_bound"], keep_default_na=False, quoting=csv.QUOTE_NONE, na_values=['NaN', 'null'], delimiter='\t')

# D:\hw\W2NER-final\ann\share13_our_split_12_look_both_side_ES-dynamic_size-F4-B4\train\w2ner_train_0.ann
# ans_split_tmp = ans_split.groupby(['file_id'])
# print('D:/hw/W2NER-main/ann/'+file_format+'/'+ann_own_path+'/w2ner_'+data_format+"_"+from_enhance_count+'.ann')

ans_split = pd.DataFrame()
# file_list = os.listdir("ann/model_share13_our_split_1_look_both_side_ES-dynamic_size-F4-B4/test")
file_list=["toe_test_0.ann","toe_test_1.ann"]
# ,"toe_test_1.ann","toe_test_2.ann"
for enhance_file in file_list:
    tmp_df = pd.read_csv("ann/cadec_19/"+data_format+"/"+enhance_file, names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
    tmp_df['entity_bound'] = tmp_df['start_end'].apply(lambda x: ",".join([x.split(',')[0],x.split(',')[-1]]))
    tmp_df = tmp_df.drop_duplicates(subset=['file_id','entity_bound'])
    ans_split = pd.concat([ans_split,tmp_df])

ans_split_df_duplicated = ans_split.duplicated(subset=['file_id','entity_bound'], keep=False)
# ans_split = ans_split[ans_split_df_duplicated].groupby(['file_id','entity_bound']).filter(lambda x: len(x) >= (len(file_list))*0.75)
ans_split = ans_split[ans_split_df_duplicated].groupby(['file_id','entity_bound']).filter(lambda x: len(x) == (len(file_list)))
ans_split = ans_split.drop_duplicates(subset=['file_id','entity_bound'])

# # ans_split = p_c.drop_duplicates(subset=['file_id','start_end'])
ans_split_tmp = ans_split.groupby(['file_id'])

# ans_split.head()
# 7896 7909
# 設定資料夾路徑
#cadec
# folder_path = 'D:/acl/data/'+file_format+'/adr/text'
#share13/share14

# 設定輸出 JSONL 檔案的路徑
# D:\acl\data\cadec\new_share13\ann_to_share13
# output_path = "D:/acl/data/cadec/new_"+file_format+"/"+ann_own_path+"_to_"+file_format+"/toe_"+file_format+"_"+data_format+"_"+to_enhance_count+"_"+output_format+".json"
main_block_size =0
ES_action_state  = 1
NES_action_state = 1
is_write =       0      # 1寫入        / 0 不寫
get_all_entity = 1     # 1取得全部實體 / 0 則否
total_sentence = 0 
add_train_random = 0            #train random 0/1 沒有/有
contain_merge_conflict_span = 0 # 0 衝突區間個體/ 1 衝突區間全部

'''
action_state
{
    0: not work
    1: work
}
'''
support_strategy = 2
'''
support_strategy
{
    0: look forward
    1: look backward
    2: look both side
}
'''
look_forward_step = 4
look_backward_step= 4

# to_enhance_count
if main_block_size==0:
    main_block_mode = "dynamic_size"
else:
    main_block_mode = str(main_block_size)
# to_enhance_count
if support_strategy==0 and (ES_action_state or NES_action_state):
    to_enhance_count += "_look_forward"
    domain_t = main_block_mode+"-"+"F"+str(look_forward_step)+"-"+"B0"
elif support_strategy==1 and (ES_action_state or NES_action_state):
    to_enhance_count += "_look_backward"
    domain_t = main_block_mode+"-"+"F0"+"-"+"B"+str(look_backward_step)
elif support_strategy==2 and (ES_action_state or NES_action_state):
    to_enhance_count += "_look_both_side"
    domain_t = main_block_mode+"-"+"F"+str(look_forward_step)+"-"+"B"+str(look_backward_step)
else:
    to_enhance_count += ""
# 7802 0  7882 1  7930 3

if ES_action_state + NES_action_state==2:
    domain_s = "_ALL"+"-"+domain_t
elif ES_action_state==1:
    domain_s = "_ES"+"-"+domain_t
elif NES_action_state==1:
    domain_s = "_NES"+"-"+domain_t
else:
    domain_s = "_None"

# output_path = "D:/hw/W2NER-final/enhance_data/"+file_format+"/"+model+"_"+data_format+"_"+to_enhance_count+domain_s+".json"
output_path = r"D:\hw\W2NER-final\enhance_data\cadec_3\w2ner_test_enhance_0_look_both_side_ALL-dynamic_size-F4-B5.json"    
# print(output_path)

# 
merge_intervals_limit = 5 # char 數為單位
entity_intervallimit = 20       # 設35會錯
# 10*5
total_entity_number = 0
avg_sentence_len = 0
total_sentence = 0
total_entity = 0
count = 0
single_entity = 0
# print(path)
with open(path)as f_ori:
    dic_f_wordlist = json.load(f_ori)
    # ff = json.load(f_ori)
    # dic_f_wordlist = {}
    # for item in ff:
    #     # print(item)
    #     word_start_end = item['word_start_end']
    #     filename = item['filename']
    #     if filename not in dic_f_wordlist:
    #         dic_f_wordlist[filename] = word_start_end
    #     else:
    #         dic_f_wordlist[filename].extend(word_start_end)

def index_label(test_list):
    pre_text = test_list[-1]
    newList = []
    textList = []
    start = 0
    for i in range(len(pre_text)):
        start =  test_list[i][0]
        p_t_s = re.findall(pattern_s,pre_text[i])
        for it,t in enumerate(p_t_s):        
            end = start + len(t)
            if t != ' ':
                textList.append(t)
                newList.append([start, end])
            start = end
    newList.append(textList)
    return newList

def search_entity_index(sentence, ran, d):
    list_ner = []
    # print(sentence, ran, d)
    for i in range(len(d[-2])):
        entity_start = d[i][0] - ran[0]
        entity_end = d[i][1] - ran[0]
        
        if i == 0:
            # re.findall(pattern,sentence[0:entity_start])
            entity_ner = len(re.findall(pattern,sentence[0:entity_start]))
            list_ner.append(entity_ner)
            
        else:
            entity_ner = entity_ner + len(re.findall(pattern,sentence[entity_record:entity_start])) + 1
            if entity_ner < len(re.findall(pattern,sentence)):
                list_ner.append(entity_ner)
        entity_record = entity_end
    
    return list_ner

def sentence_ner(diease, sentence_dict):
    global single_entity
    ans = {}
    ans_single = {}
    for d in diease:
        for k, v in sentence_dict.items():
            sentence = v[0]
            ran = k
            if ran[0]<=d[0][0] and d[-3][-1]<=ran[1]:

                list_ner = search_entity_index(sentence, ran, d)        
                if ran not in ans_single:     
                    ans_single[ran] = []
                    ans_single[ran].append({"index":list_ner, "type":d[-1]})
                else:
                    ans_single[ran].append({"index":list_ner, "type":d[-1]})    
                single_entity+=1
                break
    for k, v in sentence_dict.items():
        ner = []
        for d in diease:
            sentence = v[0]
            ran = k
            if ran[0]<=d[0][0] and d[-3][-1]<=ran[1]:
                list_ner = search_entity_index(sentence, ran, d)        
                if ran not in ans:
                    
                    ans[ran] = []
                    ans[ran].append({"index":list_ner, "type":d[-1]})
                else:
                    ans[ran].append({"index":list_ner, "type":d[-1]})    
        
    return ans

def merge_intervals(intervals):
    if not intervals:
        return []

    # Sort intervals based on the start value
    intervals.sort(key=lambda x: x[0])

    merged = [intervals[0]]
    
    
    for current in intervals[1:]:
        # Get the last interval in merged
        last_interval = merged[-1]

    
        # if current[0] - last_interval[1] <= merge_intervals_limit and (last_interval[1] - last_interval[0])<entity_intervallimit:
        if current[0] - last_interval[1] <= merge_intervals_limit:
            # Merge the intervals
            merged[-1] = [last_interval[0], max(last_interval[1], current[1])]
        else:
            # Add the current interval to merged
            merged.append(current)

    return merged

def ans_list(ans, sentence_dict, filename):
    global enhance_sentence_length
    list_dict = []
    count = 0
    for k, value in sentence_dict.items():
        v = value[0]
        span_id = value[1]
        start=int(k[0])
        end = int(k[1])
        word_list = re.findall(pattern_s, v)
        # word_index_list = word_index(word_list)
        word_index_list = []
        for word in word_list:
            end = start+len(word)
            if word.strip()!="":
                word_index_list.append([start, end])
            start=end
        # print(word_index_list)
        json_dict = {
            "sentence":[], "ner":[],
            "filename": filename, "word_start_end":word_index_list,
            "span_id":span_id
        }
        json_dict["sentence"] = re.findall(pattern, v)
        if k in ans:
            json_dict["ner"] = ans[k]
            count += len(ans[k])
        if json_dict["sentence"] != []:
            list_dict.append(json_dict)
            # if span_id==1:
            enhance_sentence_length.append(len(json_dict["sentence"]))
        if len(json_dict["sentence"])==147:
            print(filename, json_dict["word_start_end"],json_dict["sentence"])
            # print(span_id)

    return list_dict, count

if is_write:
    output_path = output_path
else:
    output_path = r"pseudo.json"


with open(output_path, 'w', encoding='utf-8') as json_file:
    all_list = []
    f_name = open("answer/"+file_format+"/id/"+data_format+".id","r").read().split("\n")
    for filename in f_name:
        # filename = "04649-004477-DISCHARGE_SUMMARY.txt"
        # filename = "00174-002042-DISCHARGE_SUMMARY.txt"
        # filename = 'LIPITOR.295'
        # file_path = folder_path+"/"+filename +".txt"
        if file_format=="cadec":
            file_path = folder_path+"/"+filename +".txt"
        else:
            file_path = folder_path+"/"+filename
        # print(folder_path)
        with open(file_path, encoding='utf-8', errors='ignore') as f:
            doc_file = f.read()
            # D:\acl\data\share2013\train\text\25844-097135-ECHO_REPORT.txt
            if main_block_mode=="dynamic_size":
                word_number = len(re.findall(pattern,doc_file))
                if word_number<=200:
                    main_block_size=7
                elif 200<word_number<=350:
                    main_block_size=9
                elif 350<word_number<=500:
                    main_block_size=11
                elif 500<word_number<=1000:
                    main_block_size=13
                elif 1000<word_number<=1350:
                    main_block_size=15
                elif 1350<word_number<=1500:
                    main_block_size=16
                elif 1500<word_number<=2000:
                    main_block_size=17
                else:
                    main_block_size=19
                # origin
                # if word_number<=150:
                #     main_block_size=4
                #     # main_block_size=7
                # elif 150<word_number<=350:
                #     main_block_size=9
                # elif 350<word_number<=500:
                #     main_block_size=11
                # elif 500<word_number<=1000:
                #     main_block_size=13
                # elif 1000<word_number<=1350:
                #     main_block_size=15
                # elif 1350<word_number<=1500:
                #     main_block_size=16
                # elif 1500<word_number<=2000:
                #     main_block_size=17
                # else:
                #     main_block_size=19
            #---------------------------------
            #正確答案
            try:
                if answer_file!="":
                    true_answers = ans_tmp.get_group(filename)
                    target_start_end = true_answers[['start_end']]
                    target_entity = true_answers[['entity']]
                    index_to_int = []
                    target_entity_list = target_entity.values.tolist()
                    target_label = true_answers[['label']].values.tolist()
                    for s_e_index, s_e in enumerate(target_start_end.values.tolist()):
                        s_e = [int(span) for span in s_e[0].split(',')]
                        index_to_int.append([s_e, target_entity_list[s_e_index][0],target_label[s_e_index][0]])
                    index_to_int = sorted(index_to_int,key=itemgetter(0))

                    items_list = []
                    item_s_e = {}
                    for item in index_to_int:
                        items = []
                        # print(item)
                        if len(item[0])>2:
                            start = 0
                            item_str = []
                            item_index = []
                            itm = item[1]
                            for index in range(0, len(item[0]), 2):     
                                end = start + (item[0][index+1] - item[0][index])
                                item_str.append(itm[start:end])
                                itm = itm[end:].strip()
                                item_index.append([item[0][index], item[0][index+1]])
                            items = item_index+[item_str]
                        else:
                            items = [item[0], [item[1]]]
                        # print(items)
                        newList = index_label(items)+[item[2]]
                        # print(newList)
                        if newList[0][0] not in item_s_e:
                            item_s_e[newList[0][0]] = [newList[0][0], newList[-2][-1]]
                        else:
                            nl = max(item_s_e[newList[0][0]][-1] ,newList[-2][-1])
                            item_s_e[newList[0][0]] = [newList[0][0], nl]
                        items_list.append(newList)
                else:
                    items_list=[]
            except:
                items_list=[]

            #---------------------------------
            if filename in ans_split_tmp.groups.keys():
                split_answers = ans_split_tmp.get_group(filename)
            # share14 可以不用
            # if filename in ans_split_tmp.groups.keys() or ((data_format=="train" or data_format=="dev" or data_format=="test") and (filename not in ans_split_tmp.groups.keys() and filename in ans_tmp.groups.keys())):
            # if filename in ans_split_tmp.groups.keys() or ((data_format=="train" or data_format=="dev") and (filename not in ans_split_tmp.groups.keys() and filename in ans_tmp.groups.keys())):
            #     try:
            #         split_answers = ans_split_tmp.get_group(filename)
            #     except:
            #         split_answers = true_answers
            #     # if data_format=="train" or data_format=="dev" or data_format=="test":
            #     if data_format=="train" or data_format=="dev":
            #         split_answers['first_start'] = split_answers['start_end'].apply(lambda x: x.split(',')[0])
            #         true_answers['first_start'] = true_answers['start_end'].apply(lambda x: x.split(',')[0])
            #         # split_answers['first_start'] = split_answers['start_end'].apply(lambda x: ",".join([x.split(',')[0],x.split(',')[-1]]))
            #         # true_answers['first_start'] = true_answers['start_end'].apply(lambda x: ",".join([x.split(',')[0],x.split(',')[-1]]))
            #         miss_data = true_answers.merge(split_answers, on=["first_start"], how='left')
            #         miss_data = miss_data[miss_data['file_id_y'].isna()].iloc[:,:6].rename(columns={
            #             "file_id_x":"file_id",
            #             "label_x":"label",
            #             "start_end_x":"start_end",
            #             "entity_x":"entity",
            #             "other_x":"other"
            #             })
            #         split_answers = split_answers.append(miss_data, ignore_index=True)
            #         split_answers = split_answers.drop_duplicates(subset=['first_start'])
            #         split_answers.drop(columns=["first_start"],inplace=True)
                

                split_start_end = split_answers[['start_end']]
                split_entity = split_answers[['entity']]
                split_index_to_int = []
                split_entity_list = split_entity.values.tolist()
                for s_e_index, s_e in enumerate(split_start_end.values.tolist()):
                    s_e = [int(span) for span in s_e[0].split(',')]
                    split_index_to_int.append([s_e, split_entity_list[s_e_index][0]])
                split_index_to_int = sorted(split_index_to_int,key=itemgetter(0))
            
                split_items_list = []
                split_item_s_e = {}
                for item in split_index_to_int:
                    item0 = [[item[0][index], item[0][index+1]] for index in range(0, len(item[0]), 2)]
                    item1 = item[1].split(',')
                    item0.append(item1)
                    if item0[0][0] not in split_item_s_e:
                        split_item_s_e[item0[0][0]] = [item0[0][0], item0[-2][-1]]
                    else:
                        nl = max(split_item_s_e[item0[0][0]][-1] ,item0[-2][-1])
                        split_item_s_e[item0[0][0]] = [item0[0][0], nl]
                    split_items_list.append(item0)
                #------------------------------------------
                new_item_s_e = []
                for k, v in split_item_s_e.items():
                    new_item_s_e.append(v)

                # new_item_se = merge_intervals(new_item_s_e)

                mergedData = []
                start, end = new_item_s_e[0]

                for pair in new_item_s_e[1:]:
                    if pair[0] <= end:
                        end = max(end, pair[1])
                    else:
                        mergedData.append([start, end])
                        start, end = pair

                # 加入最後一個合併的範圍
                mergedData.append([start, end])
                merged_data = merge_intervals(mergedData)
                if contain_merge_conflict_span:
                    mergedDataArray= np.array(merged_data)
                else:
                    mergedDataArray= np.array(mergedData)

                # --------------------------
                # pattern_s = r'[\w]+|\n|[\s]+|[^\s\w]'
                doc_file_l = list(doc_file)
                start = 0
                residual_target_dict = {}
                for value_indx, value in enumerate(merged_data):
                    end = value[0]
                    residual = "".join(doc_file_l[start:end])
                    target = "".join(doc_file_l[value[0]:value[1]])
                    residual_target_dict[2*value_indx] = residual
                    residual_target_dict[2*value_indx+1] = target
                    start = value[1]
                residual = "".join(doc_file_l[value[1]:])
                residual_target_dict[len(residual_target_dict)] = residual
                
                #-------------------------------------------------
                tmp = []
                finish_list = []
                ner = []
                total = 0
                # total_char = 0
                count = 0
                ner_count = 0
                start = 0
                end = 0
                pre_start_end = (start, end)
                sentence_dict = {}
                nest_sentence = []
                sentence_history = []
                for k,v in residual_target_dict.items():
                    residual_v = re.findall(pattern_s,v)
                    # print(start,end,k,residual_v)
                    if k%2!=0:
                        pre_len=total
                        pre_s = tmp
                        for r_v in residual_v:
                            if r_v.strip()!="":
                                total += 1
                            # total_char += len(r_v)
                            count += len(r_v)
                            tmp.append(r_v)
                        
                        end = count
                        tmp_value =  "".join(tmp)                        
                        if sentence_history!=[]:
                        
                            prev_dict = sentence_dict[(sentence_history[-1][0], sentence_history[-1][1])]

                        # if total<=10 and sentence_dict!={} and len(re.findall(pattern,prev_dict[0]))<=main_block_size:
                        if sentence_dict!={}:
                            tmp_value =  "".join(tmp)

                            prev_dict[0] = prev_dict[0]+tmp_value
                            prev_dict[1] = ES_action_state
                            prev_dict[2] = 1
                            del sentence_dict[(sentence_history[-1][0], sentence_history[-1][1])]                
                            start = sentence_history[-1][0]
                            
                            sentence_length.pop(-1)
                            total_prev_dict = len(re.findall(pattern,prev_dict[0]))

                            sentence_dict[(start, start+len(prev_dict[0]))] = prev_dict
                            sentence_length.append(len(re.findall(pattern,prev_dict[0])))
                            sentence_history.append([start, start+len(prev_dict[0])])

                            start = end
                            tmp=[]
                            total=0
                        else:
                            # print(start)
                            tmp_value =  "".join(tmp)
                            property = [tmp_value,ES_action_state,1]
                            sentence_length.append(total)
                            sentence_dict[(start, end)] = property
                            sentence_history.append([start, end])
                            tmp=[]
                            total=0
                            start = end

                    else:
                        # print(residual_v)
                        for r_v in residual_v:
                            if total >= main_block_size:

                                end = count
                                tmp_value =  "".join(tmp)
                                sentence_dict[(start, end)] = [tmp_value,NES_action_state,0]
                                sentence_history.append([start, end])
                                sentence_length.append(total)
                                finish_list.append(sentence_dict)
                                start = end                                
                                tmp=[]
                                total=0
                            if r_v.strip()!="":
                                total += 1
                            # total_char += len(r_v)
                            count += len(r_v)
                            tmp.append(r_v)
                if tmp!=[] and file_format=="cadec":
                # if tmp!=[] :
                    # print(tmp)
                    tmp_value =  "".join(tmp)
                    if tmp_value.strip()!="":
                        end = start + len(tmp_value)
                #         # print(total)
                        sentence_length.append(total)
                        sentence_dict[(start, end)] = [tmp_value,NES_action_state,0]

                pre = 0
                con = 0
                new_sentence_dict = {}
                check_prev_sentence=""
                new_sentence_history=[]
                previous_sentence_state = []
                forward_step = look_forward_step+1
                backward_step = look_backward_step+1
                for i, (k, v) in enumerate(sentence_dict.items()):
                    sentence_word = re.findall(pattern, v[0])
                    sentence_word_num = len(sentence_word)
                    if v[1] == 0:
                        con += sentence_word_num
                        new_sentence_dict[k] = [v[0],v[2]]
                        
                    else:
                        if data_format=="train" and add_train_random:
                            look_forward_step = random.randint(0,forward_step)
                            look_backward_step = random.randint(0,backward_step)

                        if support_strategy == 0:
                            pre = dic_f_wordlist[filename][max(0, con-look_forward_step)][0]
                            # new_sentence_dict[(pre, k[1])] = [doc_file[pre : k[1]],v[2]]
                            check_flag = np.logical_and(pre >= mergedDataArray[:, 0], pre < mergedDataArray[:, 1])
                            # if pre>4000:
                            #     print(check_flag.sum(-1))
                            if check_flag.sum(-1)==1:
                                conflict_span = mergedDataArray[check_flag][-1]
                                # print(conflict_span)
                                if conflict_span[0]!=0:
                                    # if previous_sentence_state[-1]==0:
                                    pre = conflict_span[-1]
                            new_sentence_dict[(pre, k[1])] = [doc_file[pre : k[1]],v[2]]

                            con += sentence_word_num
                            new_sentence_history.append([pre,  k[1]])

                        elif support_strategy == 1:

                            con += sentence_word_num
                            pre = dic_f_wordlist[filename][:con+look_backward_step][-1][-1]
                            # if i!=0:
                            #     # k[0]=pre
                            #     new_start = new_sentence_history[-1][-1]
                            # else:
                            #     new_start = 0
                            check_flag = np.logical_and(pre >= mergedDataArray[:, 0], pre < mergedDataArray[:, 1])
                            # if check_flag.sum(-1)==1:
                            # if conflict_span[0]!=0:
                            if check_flag.sum(-1)==1:
                                conflict_span = mergedDataArray[check_flag][0]
                                if conflict_span[0]!=0:
                                    pre = conflict_span[0]

                            new_sentence_dict[(k[0], pre)] = [doc_file[k[0]:pre],v[2]]
                            new_sentence_history.append([k[0], pre])

                        elif support_strategy == 2:
                            
                            # if i>0 and len(sentence_dict)>1 and sentence_word_num<7:
                            #     continue
                            # try:
                            # print(look_forward_step)
                            if look_forward_step==0:
                                pre = k[0]
                                # print(pre)
       
                            else:
                                pre = dic_f_wordlist[filename][max(0, con-look_forward_step)][0]
                                # print(pre)
                            

                            check_flag = np.logical_and( pre> mergedDataArray[:, 0], pre <= mergedDataArray[:, 1])
                            if check_flag.sum(-1)==1:
                                conflict_span = mergedDataArray[check_flag][0]
                                if conflict_span[0]!=0:
                                    
                                    # pre = conflict_span[0]
                                    if pre - conflict_span[0]<30:
                                        pre = conflict_span[0]
                                    else:
                                        pre = conflict_span[-1]
                            
                            con += sentence_word_num
                            if look_backward_step==0:
                                pre_e = k[-1]
                            else:
                                pre_e = dic_f_wordlist[filename][:con+look_backward_step][-1][-1]
                                check_flag = np.logical_and(pre_e >= mergedDataArray[:, 0], pre_e < mergedDataArray[:, 1])
                                if check_flag.sum(-1)==1:
                                    conflict_span = mergedDataArray[check_flag][-1]
                                    if conflict_span[0]!=0:
                                        
                                        # pre_e = conflict_span[-1]
                                        if conflict_span[-1] - pre_e<30:
                                            pre_e = conflict_span[-1]
                                        else:
                                            pre_e = conflict_span[0]
                            
                            
                            new_sentence_dict[(pre, pre_e)] = [doc_file[pre:pre_e],v[2]]
                            new_sentence_history.append([pre, pre_e])
                        

                    previous_sentence_state = [sentence_word_num,v[1]]
                    
                    

                ans = sentence_ner(items_list, new_sentence_dict)
                list_dict, ner_count = ans_list(ans, new_sentence_dict, filename)
                total_entity+=ner_count
                total_sentence+=len(list_dict)
                # if filename in ans_split_tmp.groups.keys() and filename in ans_tmp.groups.keys():
                # #     assert ner_count==len(target_entity)
                #     if ner_count!=len(target_entity):
                #         print(filename)
                all_list.extend(list_dict)
                
            #正確答案(要放ner)/沒預測答案(全切往前看5)
            elif filename not in ans_split_tmp.groups.keys():
                # print(filename)

                sentence_dict = {}
                for i in range(0,len(dic_f_wordlist[filename]),main_block_size):
                    # start_end= dic_f_wordlist[filename][i:i+main_block_size]
                    start_end= dic_f_wordlist[filename][min(i,max(i-look_forward_step,0)):i+main_block_size+look_backward_step]
                    if len(start_end)>5: 
                        # print(start_end)
                        sentence = doc_file[start_end[0][0]:start_end[-1][-1]]
                        sentence_dict[(start_end[0][0], start_end[-1][-1])] = [sentence,0]
                #--------------------------------------------------------------
                total_sentence+=len(sentence_dict)
                ans = sentence_ner(items_list, sentence_dict)
                list_dict, ner_count = ans_list(ans, sentence_dict, filename)
                total_entity+=ner_count
                # print(ner_count)
                # if filename in ans_tmp.groups.keys():
                #     assert ner_count==len(target_entity)
                all_list.extend(list_dict)
        # break
    json.dump(all_list, json_file, ensure_ascii=False)
import statistics
total_sentence,total_entity,single_entity,max(enhance_sentence_length),min(enhance_sentence_length),statistics.mean(enhance_sentence_length),enhance_sentence_length
            

(2133,
 1457,
 986,
 90,
 3,
 15.958274730426629,
 [11,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  14,
  7,
  11,
  36,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  8,
  11,
  15,
  15,
  15,
  15,
  15,
  10,
  13,
  36,
  17,
  23,
  17,
  17,
  18,
  17,
  17,
  17,
  25,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  12,
  11,
  15,
  15,
  15,
  20,
  15,
  15,
  15,
  15,
  15,
  18,
  15,
  15,
  15,
  15,
  9,
  11,
  14,
  7,
  7,
  7,
  14,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  8,
  6,
  13,
  15,
  15,
  15,
  28,
  15,
  23,
  15,
  15,
  20,
  22,
  15,
  17,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  11,
  11,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  8,
  11,
  15,
  15,
  15,
  15,
  15,
  15,
  10,
  11,
  15,
  19,
  31,
  21,
  22,
  15,
  15,
  15,
  11,
  8,
  21,
  15,


In [3]:
import pandas as pd
import os
file_format = "cadec"
ans_data_path="test"
ans_split = pd.DataFrame()
# folder = "share13_our_split_2"
folder = "cadec_19"
# file_list = os.listdir("ann/"+folder+"/"+ans_data_path)
file_list = ['w2ner_test_0.ann','w2ner_test_1.ann']
for enhance_file in file_list:
    tmp_df = pd.read_csv("ann/"+folder+"/"+ans_data_path+"/"+enhance_file, names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
    # tmp_df = pd.read_csv("D:/hw/W2NER-main/ann/share14/ann/"+enhance_file, names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
    # D:\hw\W2NER-main\ann\share14\ann\w2ner_test_123.ann
    tmp_df['entity_bound'] = tmp_df['start_end'].apply(lambda x: ",".join([x.split(',')[0],x.split(',')[-1]]))
    tmp_df = tmp_df.drop_duplicates(subset=['file_id','entity_bound'])

    ans_split = pd.concat([ans_split,tmp_df])

ans_split_df_duplicated = ans_split.duplicated(subset=['file_id','entity_bound'], keep=False)
# ans_split = ans_split[ans_split_df_duplicated].groupby(['file_id','entity_bound']).filter(lambda x: len(x) >= (len(file_list))*0.75)
ans_split = ans_split[ans_split_df_duplicated].groupby(['file_id','entity_bound']).filter(lambda x: len(x) == (len(file_list)))
ans_split = ans_split.drop_duplicates(subset=['file_id','entity_bound'])
# ans_split
# answer_file='acl/data/'+file_format+'/'+ans_data_path+'.ann'
# answer_file='acl/data/cadec/adr/ann'
# ans = pd.read_csv(answer_file, names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
# ans['entity_bound'] = ans['start_end'].apply(lambda x: ",".join([x.split(',')[0],x.split(',')[-1]]))
len(ans_split)


549

In [4]:
tmp_df = pd.read_csv("ann/"+folder+"/"+ans_data_path+"/"+enhance_file, names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
tmp_df

,file_id,label,start_end,entity,other
0,ARTHROTEC.102,adr,"187,190,190,191,191,192,193,198,199,201,202,206","can,',t,stand,or,walk",
1,ARTHROTEC.104,adr,"5,13",diarrhea,
2,ARTHROTEC.104,adr,"20,32",constipation,
3,ARTHROTEC.104,adr,"34,41",fatigue,
4,ARTHROTEC.104,adr,"161,167,168,182,183,185,186,189,190,195","severe,osteoarthritis,in,the,knees",
...,...,...,...,...,...
1005,VOLTAREN.8,adr,"392,402",hoarseness,
1006,VOLTAREN.8,adr,"403,411",coughing,
1007,VOLTAREN.8,adr,"413,418,419,425","heart,racing",
1008,VOLTAREN.8,adr,"427,438",disoriented,


In [18]:
# pd.merge(ans_split,ans,how="inner",on=['file_id','entity_bound'])
# .iloc[:,:6].rename(columns={"label_x":"label","start_end_x":"start_end","entity_x":"entity"})
# ans
x = pd.merge(ans_split,ans,how="left",on=['file_id','entity_bound'])
w = x[x['label_y'].isna()].iloc[:,:5].rename(columns={"label_x":"label","start_end_x":"start_end","entity_x":"entity"})
len(w)
# w.to_csv("error_predict.ann", sep='\t', index=False,header=None)
# ans_split

176

In [4]:
# p_1 = pd.read_csv("ann/"+folder+"/test/w2ner_test_0.ann", names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
# p_2 = pd.read_csv("ann/"+folder+"/test/w2ner_test_1.ann", names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
# p_3 = pd.read_csv("ann/"+folder+"/test/w2ner_test_2.ann", names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
# p_4 = pd.read_csv("ann/"+folder+"/test/w2ner_test_3.ann", names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
# p_5 = pd.read_csv("ann/"+folder+"/test/w2ner_test_4.ann", names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
# p_c = pd.concat([p_1,p_2,p_3,p_4])
p_i = pd.merge(p_1,p_2,how="inner")
# p_i = pd.merge(p_i,p_3,how="inner")
# p_i = pd.merge(p_i,p_4,how="inner")
# p_i = pd.merge(p_i,p_5,how="inner")
p_i['entity_bound'] = p_i['start_end'].apply(lambda x: ",".join([x.split(',')[0],x.split(',')[-1]]))
# ans_split = p_c.drop_duplicates(subset=['file_id','start_end'])
# ans_split_tmp = p_i.groupby(['file_id'])

In [5]:
pd.merge(p_i,ans,how="inner",on=['file_id','entity_bound']).iloc[:,:6].rename(columns={"label_x":"label","start_end_x":"start_end","entity_x":"entity"})
# p_i

,file_id,label,start_end,entity,other_x,entity_bound
0,00176-102920-ECHO_REPORT.txt,disorder,"120,131,132,140","Pericardial,effusion",,"120,140"
1,00176-102920-ECHO_REPORT.txt,disorder,"430,434,435,441,442,444,445,455,456,463","left,atrium,is,moderately,dilated",,"430,463"
2,00176-102920-ECHO_REPORT.txt,disorder,"534,539,540,546,547,549,550,560,561,568","right,atrium,is,moderately,dilated",,"534,568"
3,00176-102920-ECHO_REPORT.txt,disorder,"614,618,619,630,631,642","left,ventricular,hypertrophy",,"614,642"
4,00176-102920-ECHO_REPORT.txt,disorder,"809,813,814,825,826,830,831,837,838,851","left,ventricular,wall,motion,abnormalities",,"809,851"
...,...,...,...,...,...,...
4119,26522-011368-DISCHARGE_SUMMARY.txt,disorder,"6661,6671",hemorrhage,,"6661,6671"
4120,26522-011368-DISCHARGE_SUMMARY.txt,disorder,"6849,6856,6857,6859","dropped,BP",,"6849,6859"
4121,26522-011368-DISCHARGE_SUMMARY.txt,disorder,"7428,7436,7437,7439","dropping,BP",,"7428,7439"
4122,26522-011368-DISCHARGE_SUMMARY.txt,disorder,"7452,7459,7460,7470","cardiac,arrhythmia",,"7452,7470"


In [28]:
doc_file

'Terrible short term memory problems, lack of motivation, weak and wobbly unbalanced feeling (walked like a 90 year old), not sure footed as I walked, overall weak feeling, difficulty driving due to muscle function irregularities, tightness in throat with occasional choking problem, lightheaded foggy brain (like being in a stupor), loss of endurance, heavy legs and arms, eye muscle spasms, dizziness (vertigo), lousy sleeping at night, terrible concentration, bad dreams nightly, foot and leg cramps especially at night in bed, always exhausted, hip joint pain, muscular aches, swallowing problems, hard to get out of bed in the morning, constant muscle tension in legs (inflexibility), sadness, mood swings, loss of muscle mass in chest and arms.\nI took 10 mg/day of Lipitor for 6 years.\nIt worked perfectly and made my blood test numbers for cholesterol, HDL/LDL and Triglycerides look textbook perfect.\nA few of the symptoms I listed (mostly muscular) quickly appeared when I began the Lipit

In [21]:
residual_target_dict

{0: '',
 1: 'Terrible short term memory problems, lack of motivation, weak and wobbly unbalanced feeling (walked like a 90 year old), not sure footed as I walked, overall weak feeling, difficulty driving due to muscle function irregularities, tightness in throat with occasional choking problem, lightheaded foggy brain (like being in a stupor), loss of endurance, heavy legs and arms, eye muscle spasms, dizziness (vertigo), lousy sleeping at night, terrible concentration, bad dreams nightly, foot and leg cramps especially at night in bed, always exhausted, hip joint pain, muscular aches, swallowing problems, hard to get out of bed in the morning, constant muscle tension in legs (inflexibility), sadness, mood swings, loss of muscle mass in chest and arms',
 2: '.\nI took 10 mg/day of Lipitor for 6 years.\nIt worked perfectly and made my blood test numbers for cholesterol, HDL/LDL and Triglycerides look textbook perfect.\nA few of the symptoms I listed (mostly muscular) quickly appeared wh

In [23]:
split_answers

,file_id,label,start_end,entity,other
286,LIPITOR.295,diease,"0,8,9,14,15,19,20,26,27,35","Terrible,short,term,memory,problems",
287,LIPITOR.295,diease,"9,14,15,19,20,26,27,35","short,term,memory,problems",
288,LIPITOR.295,diease,"37,41,42,44,45,55,55,56,57,61,62,65,66,72,73,8...","lack,of,motivation,,,weak,and,wobbly,unbalance...",
289,LIPITOR.295,diease,"37,41,42,44,45,55,55,56,57,61,62,65,66,72,73,8...","lack,of,motivation,,,weak,and,wobbly,unbalance...",
290,LIPITOR.295,diease,"37,41,42,44,45,55,55,56,57,61,62,65,66,72,73,8...","lack,of,motivation,,,weak,and,wobbly,unbalance...",
...,...,...,...,...,...
352,LIPITOR.295,diease,"640,648,649,655,656,663,664,666,667,671,672,67...","constant,muscle,tension,in,legs,(,inflexibility,)",
353,LIPITOR.295,diease,"689,696",sadness,
354,LIPITOR.295,diease,"698,702,703,709","mood,swings",
355,LIPITOR.295,diease,"711,715,716,718,719,725,726,730,731,733,734,73...","loss,of,muscle,mass,in,chest,and,arms",


In [25]:
true_answers

,file_id,label,start_end,entity,other
1963,LIPITOR.295,ADR,"0,35",Terrible short term memory problems,
1964,LIPITOR.295,ADR,"37,55",lack of motivation,
1965,LIPITOR.295,ADR,"57,83",weak and wobbly unbalanced,
1966,LIPITOR.295,ADR,"93,118",walked like a 90 year old,
1967,LIPITOR.295,ADR,"121,148",not sure footed as I walked,
1968,LIPITOR.295,ADR,"150,170",overall weak feeling,
1969,LIPITOR.295,ADR,"198,228",muscle function irregularities,
1970,LIPITOR.295,ADR,"230,281",tightness in throat with occasional choking pr...,
1971,LIPITOR.295,ADR,"283,331",lightheaded foggy brain (like being in a stupor),
1972,LIPITOR.295,ADR,"333,350",loss of endurance,


In [8]:

import pandas as pd
import os
ans_split = pd.DataFrame()
# file_list = os.listdir("/data/m11115088/W2NER_final/ann/share14/"+data_format)
file_list = os.listdir("ann/share14_our_split_1_new_inner_look_both_side_ES-dynamic_size-F3-B3/test")
for enhance_file in file_list:
    tmp_df = pd.read_csv("ann/share14_our_split_1_new_inner_look_both_side_ES-dynamic_size-F3-B3/test/"+enhance_file, names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
    tmp_df['entity_bound'] = tmp_df['start_end'].apply(lambda x: ",".join([x.split(',')[0],x.split(',')[-1]]))
    ans_split = pd.concat([ans_split,tmp_df])

ans_split_df_duplicated = ans_split.duplicated(subset=['file_id','entity_bound'], keep=False)
# ans_split = ans_split[ans_split_df_duplicated].groupby(['file_id','entity_bound']).filter(lambda x: len(x) >= (len(file_list))*0.75)
# ans_split = ans_split.drop_duplicates(subset=['file_id','entity_bound'])
ans_split[ans_split_df_duplicated]

,file_id,label,start_end,entity,other,entity_bound
1933,06828-026269-DISCHARGE_SUMMARY.txt,disorder,"2354,2361,2370,2374,2375,2379","Chronic,back,pain",,"2354,2379"
1934,06828-026269-DISCHARGE_SUMMARY.txt,disorder,"2354,2361,2362,2369,2375,2379","Chronic,buttock,pain",,"2354,2379"
5713,16737-003522-DISCHARGE_SUMMARY.txt,disorder,"6257,6261,6268,6274,6275,6283","open,fibula,fracture",,"6257,6283"
5714,16737-003522-DISCHARGE_SUMMARY.txt,disorder,"6257,6261,6262,6267,6275,6283","open,tibia,fracture",,"6257,6283"


In [10]:
"{:3.4s}: total sentence: {:3.0f} total entity: {:3f}".format("0",1,2)


'0  : total sentence:   1 total entity: 2.000000'